### Importing Source Files

In [0]:
%run "../Includes/Configuration"

In [0]:
%run "../Includes/Common_Functions"

In [0]:
%run "../Includes/sas alternative"

In [0]:
df = spark.read.parquet(f"{Presentation_path}/Race_Results_R")

In [0]:
df.display()

In [0]:
# find out total points based on race_year, driver_name, driver_nationality, team
from pyspark.sql.functions import sum,col,count,when
df_total_points_count_positionTrue = df.groupBy('race_year','driver_name', 'driver_nationality', 'team').agg(sum('points').alias("total_points"),count(when(col("position")==1, True)).alias("wins"))

In [0]:
df_total_points_count_positionTrue.display()

In [0]:
df_total_points_count_positionTrue.filter(col('race_year')== 2020).display()

In [0]:
# identify the rank based up on partitionBY(race_year) order by wins and total points descending order (window functions)

from pyspark.sql.window import Window
from pyspark.sql.functions import rank,desc

window_spec = Window.partitionBy("race_year").orderBy(desc("wins"),desc("total_points"))

In [0]:
df_window = df_total_points_count_positionTrue.withColumn("rank", rank().over(window_spec))
df_window.display()

In [0]:
from pyspark.sql.functions import dense_rank
df_window_dense = df_total_points_count_positionTrue.withColumn("dense_rank", dense_rank().over(window_spec))
df_window_dense.display()

In [0]:
# race_year 2020 display
df_window.filter(col('race_year')==2020).display()

In [0]:
df_window.write.mode('overwrite').parquet(f"{Presentation_path}/Driver_Standing")

In [0]:
dbutils.notebook.exit("Sucessed")